<a href="https://colab.research.google.com/github/luizinfpp/python-notebooks/blob/sentiment-analysis/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#Gathering data

In [2]:
data_neg = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment/neg_tweets.txt", header=None)
data_pos = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment/pos_tweets.txt", header=None)

In [3]:
data_neg = data_neg.rename(columns={0: 'tweet'})
data_pos = data_pos.rename(columns={0: 'tweet'})

In [4]:
data_neg["label"] = "neg"
data_pos["label"] = "pos"

In [5]:
data_train = pd.concat([data_neg.iloc[:500], data_pos.iloc[:500]])
data_train = data_train.reset_index(drop=True)

#Modelling

In [6]:
vectorizer = CountVectorizer(analyzer = 'word', lowercase = False)
features = vectorizer.fit_transform(data_train['tweet'].values)
features_nd = features.toarray() 

##Train

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        data_train['label'].values,
        train_size=0.8, 
        random_state=1234)

##Model

In [8]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)

##Accuracy

In [9]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.77
